# Data Preparation

### Data preparation: select, clean and construct data

- Impute missing values
- Select main input variables X (feature engineering)
- Decide how to handle correlated input features

In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.feature_selection import chi2, VarianceThreshold
import sklearn.linear_model
import seaborn as sns

# supressing warnings for readability
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# To plot pretty figures directly within Jupyter
%matplotlib inline

# choose your own style: https://matplotlib.org/3.1.0/gallery/style_sheets/style_sheets_reference.html
# plt.style.use("seaborn-whitegrid")

# setting up plot style 
plt.style.use('seaborn-poster')
plt.style.use('fivethirtyeight')

# Go to town with https://matplotlib.org/tutorials/introductory/customizing.html
# plt.rcParams.keys()
mpl.rc("axes", labelsize=14, titlesize=14)
mpl.rc("figure", titlesize=20)
mpl.rc("xtick", labelsize=12)
mpl.rc("ytick", labelsize=12)

# contants for figsize
S = (5, 5)
M = (8, 8)
L = (12, 12)

In [7]:
# helper function to count top 10 unique values for dataframe
# pd.concat takes as list of Series or DataFrames to concatenate
# make this list using a list comprehension
def count_values(df):
  _value_counts = [df[col].value_counts().sort_values(ascending=False).head(10) for col in df.columns]
  return pd.concat(_value_counts, axis=1).transpose()

In [215]:
# import data
df_client = pd.read_csv('../data/external/train/client_train.csv')
df_invoice = pd.read_csv('../data/external/train/invoice_train.csv')

### Impute missing values

#### Client Dataset

This dataset seems to not have any missing values

In [216]:
df_client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   disrict        135493 non-null  int64  
 1   client_id      135493 non-null  object 
 2   client_catg    135493 non-null  int64  
 3   region         135493 non-null  int64  
 4   creation_date  135493 non-null  object 
 5   target         135493 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 6.2+ MB


In [217]:
df_client.head()

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.000
1,69,train_Client_1,11,107,29/05/2002,0.000
2,62,train_Client_10,11,301,13/03/1986,0.000
3,69,train_Client_100,11,105,11/07/1996,0.000
4,62,train_Client_1000,11,303,14/10/2014,0.000


In [218]:
dfc = df_client.copy()

In [219]:
dfc.rename(columns={'disrict': 'district', 'client_catg': 'client_cat'}, inplace=True)
dfc.head()

,district,client_id,client_cat,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.000
1,69,train_Client_1,11,107,29/05/2002,0.000
2,62,train_Client_10,11,301,13/03/1986,0.000
3,69,train_Client_100,11,105,11/07/1996,0.000
4,62,train_Client_1000,11,303,14/10/2014,0.000


#### Feature Engineer

In [220]:
cat_cols = ['district', 'client_cat', 'region']
dt_cols = ['creation_date']
bool_cols = ['target']

def convert_dtype_client(df, cat=[], num=[], dt=[], bool=[]):
    convert_dict = {}
    for col in cat_cols:
        convert_dict[col] = 'category'
    for col in bool_cols:
        convert_dict[col] = 'bool'
    df = df.astype(convert_dict)
    df[num] = df[num].apply(pd.to_numeric)
    df[dt] = df[dt].apply(pd.to_datetime)
    return df

dfc = convert_dtype(dfc, cat=cat_cols, dt=dt_cols, bool=bool_cols)
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   district       135493 non-null  category      
 1   client_id      135493 non-null  object        
 2   client_cat     135493 non-null  category      
 3   region         135493 non-null  category      
 4   creation_date  135493 non-null  datetime64[ns]
 5   target         135493 non-null  bool          
dtypes: bool(1), category(3), datetime64[ns](1), object(1)
memory usage: 2.6+ MB


In [221]:
dfc.to_csv('../data/processed/client_train.csv')

#### Invoice Dataset

This dataset also seems to have no missing values

In [174]:
df_invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4476749 entries, 0 to 4476748
Data columns (total 16 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   client_id             object
 1   invoice_date          object
 2   tarif_type            int64 
 3   counter_number        int64 
 4   counter_statue        object
 5   counter_code          int64 
 6   reading_remarque      int64 
 7   counter_coefficient   int64 
 8   consommation_level_1  int64 
 9   consommation_level_2  int64 
 10  consommation_level_3  int64 
 11  consommation_level_4  int64 
 12  old_index             int64 
 13  new_index             int64 
 14  months_number         int64 
 15  counter_type          object
dtypes: int64(12), object(4)
memory usage: 546.5+ MB


In [175]:
dfc = df_invoice.copy()

In [176]:
dfc.isnull().value_counts()

client_id  invoice_date  tarif_type  counter_number  counter_statue  counter_code  reading_remarque  counter_coefficient  consommation_level_1  consommation_level_2  consommation_level_3  consommation_level_4  old_index  new_index  months_number  counter_type
False      False         False       False           False           False         False             False                False                 False                 False                 False                 False      False      False          False           4476749
Name: count, dtype: int64

In [177]:
dfc.rename(columns={'counter_statue': 'counter_status', 'reading_remarque': 'counter_score', 'months_number': 'month'}, inplace=True)
dfc.head()

,client_id,invoice_date,tarif_type,counter_number,counter_status,counter_code,counter_score,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,month,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC


#### Feature Engineering

In [178]:
def remove_rows(df):
    print('Rows before removal:', len(df))
    df = df.drop(df[df['counter_status'].isin([769, 618, 269375, 46, 420, 'A'])].index)
    df = df.drop(df[~df['month'].isin(range(1, 13))].index)
    print('Rows after removal:', len(df))
    return df

# def remove_outliers(df):
#     print('Rows before removal:', len(df))
#     df = df.drop(df[df['counter_status'].isin([769, 618, 269375, 46, 420, 'A'])].index)
#     df = df.drop(df[~df['month'].isin(range(1, 13))].index)
#     print('Rows after removal:', len(df))
#     return df

def calculate_outlier_threshold(column):
    # Calculate the first quartile (Q1) and third quartile (Q3)
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    
    # Calculate the interquartile range (IQR)
    IQR = Q3 - Q1

    # Calculate the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    print('(lower, upper):', (lower_bound, upper_bound))
    return lower_bound, upper_bound

def bin_data(df, column, num_bins):
    # Create bins
    edge, bins = pd.qcut(df[column], q=num_bins, retbins=True)
    # labels = [str(interval) for interval in bins.categories]

    # Assign labels to each bin
    labels = []
    for x in range(1, len(bins)):
        labels.append(f'{bins[x-1]} - {bins[x]}')

    # # Create the new categorical column based on the bin labels
    return pd.qcut(df[column], q=num_bins, labels=labels)

In [179]:
len(remove_rows(dfc)) / len(dfc) * 100

Rows before removal: 4476749
Rows after removal: 4452681


99.46237772097565

In [180]:
# For now, we will not remove any outliers
# since based on EDA we did, fraud have some correlation
# with outliers, especially with consommation level
dfc = remove_rows(dfc)

Rows before removal: 4476749
Rows after removal: 4452681


In [181]:
# dfc.counter_status.value_counts()
dfc['counter_status'] = pd.to_numeric(dfc['counter_status'], errors='coerce')
dfc.counter_status.value_counts()

counter_status
0    4355396
1      73939
5      20372
4       2689
3        257
2         28
Name: count, dtype: int64

In [182]:
len(dfc[dfc['old_index'] == dfc['new_index']]) / len(dfc) * 100

10.505603253410698

In [183]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4452681 entries, 0 to 4476748
Data columns (total 16 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   client_id             object
 1   invoice_date          object
 2   tarif_type            int64 
 3   counter_number        int64 
 4   counter_status        int64 
 5   counter_code          int64 
 6   counter_score         int64 
 7   counter_coefficient   int64 
 8   consommation_level_1  int64 
 9   consommation_level_2  int64 
 10  consommation_level_3  int64 
 11  consommation_level_4  int64 
 12  old_index             int64 
 13  new_index             int64 
 14  month                 int64 
 15  counter_type          object
dtypes: int64(13), object(3)
memory usage: 577.5+ MB


In [184]:
cat_cols = ['tarif_type', 'counter_status', 'counter_code', 'month', 'counter_type']
dt_cols = ['invoice_date']
num_cols = ['counter_number', 'counter_score', 'counter_coefficient', 'consommation_level_1', 'consommation_level_2', 'consommation_level_3', 'consommation_level_4', 'old_index', 'new_index']

def convert_dtype_invoice(df, cat=[], num=[], dt=[], bool=[]):
    convert_dict = {}
    for col in cat:
        convert_dict[col] = 'category'
    for col in bool:
        convert_dict[col] = 'bool'
    df = df.astype(convert_dict)
    df[num] = df[num].apply(pd.to_numeric)
    df[dt] = df[dt].apply(pd.to_datetime)
    return df

dfc = convert_dtype_invoice(dfc, cat=cat_cols, num=num_cols, dt=dt_cols)

In [185]:
dfc.describe()

,invoice_date,counter_number,counter_score,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index
count,4452681,4452681.000,4452681.000,4452681.000,4452681.000,4452681.000,4452681.000,4452681.000,4452681.000,4452681.000
mean,2013-03-15 09:57:52.733752064,123106525081.797,7.318,1.003,405.475,107.991,19.453,52.423,17776.222,18312.730
min,1977-06-09 00:00:00,0.000,6.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,2010-01-15 00:00:00,121108.000,6.000,1.000,80.000,0.000,0.000,0.000,1793.000,2052.000
50%,2013-07-05 00:00:00,494482.000,8.000,1.000,273.000,0.000,0.000,0.000,7690.000,8175.000
75%,2016-09-02 00:00:00,1114692.000,9.000,1.000,600.000,0.000,0.000,0.000,21659.000,22310.000
max,2019-12-07 00:00:00,27981145458733.000,9.000,50.000,999910.000,999073.000,2400.000,547946.000,2800280.000,2870972.000
std,NaN,1657587062423.553,1.360,0.309,728.804,1214.701,121.403,860.720,40427.602,40910.476


In [186]:
# reduce memory: strings to categories

# int64 to smallest, unsigned int
for col in dfc.select_dtypes(include='int64').columns:
    dfc[col] = pd.to_numeric(dfc[col], downcast='unsigned')

for col in dfc.select_dtypes(include='object').columns:
    dfc[col] = dfc[col].astype('category')

dfc.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 4452681 entries, 0 to 4476748
Columns: 16 entries, client_id to counter_type
dtypes: category(6), datetime64[ns](1), uint16(1), uint32(5), uint64(1), uint8(2)
memory usage: 247.1 MB


In [212]:
def check_same_month(row):
    if row['invoice_date'].month == row['month']:
        return True
    else:
        return False

def add_feature_invoice(df):
    df = df.copy()
    conso_cols = ['consommation_level_1', 'consommation_level_2', 'consommation_level_3', 'consommation_level_4']
    conso_cols_not1 = ['consommation_level_2', 'consommation_level_3', 'consommation_level_4']
    df['consommation_total_not1'] = df[conso_cols_not1].sum(axis=1)
    df['consommation_total'] = df[conso_cols].sum(axis=1)
    df['bin_counter_number'] = bin_data(df, 'counter_number', 10)
    df['bin_cons_1'] = bin_data(df, 'consommation_level_1', 9)
    above_100 = conso_cols_not1 + ['consommation_total']
    for x in above_100:
        col = df[x].copy()
        col = col.map(lambda x: True if x > 100 else False)
        df[x + 'above100'] = col
    df['same_month'] = df.apply(check_same_month, axis=1)
    return df

In [210]:
dfc = add_feature_invoice(dfc)

In [211]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4452681 entries, 0 to 4476748
Data columns (total 25 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   client_id                     category      
 1   invoice_date                  datetime64[ns]
 2   tarif_type                    category      
 3   counter_number                uint64        
 4   counter_status                category      
 5   counter_code                  category      
 6   counter_score                 uint8         
 7   counter_coefficient           uint8         
 8   consommation_level_1          uint32        
 9   consommation_level_2          uint32        
 10  consommation_level_3          uint16        
 11  consommation_level_4          uint32        
 12  old_index                     uint32        
 13  new_index                     uint32        
 14  month                         category      
 15  counter_type                  categor

In [213]:
# reduce memory: strings to categories

# int64 to smallest, unsigned int
for col in dfc.select_dtypes(include='int64').columns:
    dfc[col] = pd.to_numeric(dfc[col], downcast='unsigned')

for col in dfc.select_dtypes(include='object').columns:
    dfc[col] = dfc[col].astype('category')

dfc.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 4452681 entries, 0 to 4476748
Columns: 25 entries, client_id to same_month
dtypes: bool(5), category(8), datetime64[ns](1), uint16(1), uint32(7), uint64(1), uint8(2)
memory usage: 310.8 MB


In [214]:
dfc.to_csv('../data/processed/invoice_train.csv')

### Merged Dataset

In [2]:
# import data
df_client = pd.read_csv('../data/processed/client_train.csv')
df_invoice = pd.read_csv('../data/processed/invoice_train.csv')

In [3]:
dfc = pd.merge(df_client, df_invoice, on='client_id')

In [4]:
dfc.head()

,Unnamed: 0_x,district,client_id,client_cat,region,creation_date,target,Unnamed: 0_y,invoice_date,tarif_type,...,counter_type,consommation_total_not1,consommation_total,bin_counter_number,bin_cons_1,consommation_level_2above100,consommation_level_3above100,consommation_level_4above100,consommation_totalabove100,same_month
0,0,60,train_Client_0,11,101,1994-12-31,False,0,2014-03-24,11,...,ELEC,0,82,1321324.0 - 6787407.0,64.0 - 132.0,False,False,False,False,False
1,0,60,train_Client_0,11,101,1994-12-31,False,1,2013-03-29,11,...,ELEC,184,1384,1321324.0 - 6787407.0,810.0 - 999910.0,True,False,False,True,False
2,0,60,train_Client_0,11,101,1994-12-31,False,2,2015-03-23,11,...,ELEC,0,123,1321324.0 - 6787407.0,64.0 - 132.0,False,False,False,True,False
3,0,60,train_Client_0,11,101,1994-12-31,False,3,2015-07-13,11,...,ELEC,0,102,1321324.0 - 6787407.0,64.0 - 132.0,False,False,False,True,False
4,0,60,train_Client_0,11,101,1994-12-31,False,4,2016-11-17,11,...,ELEC,0,572,1321324.0 - 6787407.0,467.0 - 655.0,False,False,False,True,False


In [5]:
dfc.to_csv('../data/processed/merged_train.csv')